In [1]:
import json
import os
import pickle
import re
from typing import List
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [10]:
def get_all_links_with_depth(
    start_url: str, domain_filter: str = "https://docs.lammps.org/", max_depth: int = 2
):
    to_visit = [(start_url, 0)]

    visited = set([start_url])

    all_links = []

    while to_visit:
        current_url, current_depth = to_visit.pop(0)
        all_links.append(current_url)

        if current_depth >= max_depth:
            continue

        try:
            response = requests.get(current_url)
            response.raise_for_status()
        except Exception as e:
            print(f"Error fetching {current_url}: {e}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        for link_tag in soup.find_all("a", href=True):
            next_link = urljoin(current_url, link_tag["href"])
            if next_link.startswith(domain_filter) and next_link not in visited:
                visited.add(next_link)
                to_visit.append((next_link, current_depth + 1))

    return all_links


In [11]:
max_depth = 1
all_links = get_all_links_with_depth(
    start_url="https://docs.lammps.org/Manual.html",
    domain_filter="https://docs.lammps.org/",
    max_depth=max_depth,
)

all_docs = []
for i, link in enumerate(all_links):
    loader = UnstructuredURLLoader(urls=[link])
    try:
        docs = loader.load()
        all_docs.extend(docs)
    except Exception as e:
        print(f"Error loading {link}: {e}")
    print(f"Loaded {i+1}/{len(all_links)}: {link}")
with open("all_docs.pkl", "wb") as f:
    pickle.dump(all_docs, f)
print("all_docs is saved to 'all_docs.pkl'.")

Loaded 1/168: https://docs.lammps.org/Manual.html
Loaded 2/168: https://docs.lammps.org/Intro.html
Loaded 3/168: https://docs.lammps.org/Install.html
Loaded 4/168: https://docs.lammps.org/Build.html
Loaded 5/168: https://docs.lammps.org/Run_head.html
Loaded 6/168: https://docs.lammps.org/Commands.html
Loaded 7/168: https://docs.lammps.org/Packages.html
Loaded 8/168: https://docs.lammps.org/Speed.html
Loaded 9/168: https://docs.lammps.org/Howto.html
Loaded 10/168: https://docs.lammps.org/Examples.html
Loaded 11/168: https://docs.lammps.org/Tools.html
Loaded 12/168: https://docs.lammps.org/Errors.html
Loaded 13/168: https://docs.lammps.org/Library.html
Loaded 14/168: https://docs.lammps.org/Python_head.html
Loaded 15/168: https://docs.lammps.org/Modify.html
Loaded 16/168: https://docs.lammps.org/Developer.html
Loaded 17/168: https://docs.lammps.org/commands_list.html
Loaded 18/168: https://docs.lammps.org/fixes.html
Loaded 19/168: https://docs.lammps.org/computes.html
Loaded 20/168: http

Error fetching or processing https://docs.lammps.org/Manual.pdf, exception: partition_pdf() is not available because one or more dependencies are not installed. Use: pip install "unstructured[pdf]" (including quotes) to install the required dependencies


Loaded 168/168: https://docs.lammps.org/Manual.pdf
all_docs is saved to 'all_docs.pkl'.


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(all_docs)

In [13]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
vectordb = Chroma.from_documents(
    split_docs,
    embedding=embeddings,
    collection_name="lammps_manual",
    persist_directory="chroma_db",
)

In [14]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"])


In [15]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "LAMMPSはどのような分子動力学シミュレーションが可能ですか？"
answer = qa_chain.run(query)
print(answer)


LAMMPSは、液体、固体、気体状態の粒子の集団をモデル化する古典的な分子動力学（MD）コードです。原子、ポリマー、生体分子、固体（金属、セラミック、酸化物）、粒状、粗視化、あるいは巨視的なシステムを、多様な相互原子ポテンシャル（力場）や境界条件を使用してモデル化することができます。2次元または3次元のシステムを数個の粒子から数十億個の粒子までモデル化することが可能です。また、個々の粒子は原子、分子、電子、粗視化された原子群、またはメソスコピックや巨視的な物質の塊であることがあります。

LAMMPSは、並列計算機用に設計されており、MPIメッセージパッシングライブラリをサポートする任意の並列機でシリアルまたは並列で動作します。シェアードメモリのマルチコア、マルチCPUサーバー、および分散メモリクラスターやスーパーコンピューターで実行可能です。LAMMPSの一部はOpenMPマルチスレッド化、ベクトル化、GPUアクセラレーションもサポートしています。


In [23]:
def score_answers_by_docs(
    evaluator_llm: ChatOpenAI,
    questions: List[str],
    answers: List[str],
    retriever,  # ここで "evaluation_retriever" を想定
    top_k: int = 3,
):
    """
    質問リスト + 回答リストを渡すと、retriever で top_k のドキュメントを取得し、
    LLMに「回答とドキュメントは整合しているか？」をスコアリングさせる
    """
    results = []
    for i, (q, ans) in enumerate(zip(questions, answers)):
        # 関連ドキュメントを取得
        docs = retriever.get_relevant_documents(q)[:top_k]
        doc_texts = "\n\n---\n\n".join(d.page_content for d in docs)

        eval_prompt = f"""
私は、以下の「質問」と「回答」が、参照ドキュメントとどの程度整合しているかを評価します。

【質問】
{q}

【回答】
{ans}

【参照ドキュメント (上位 {top_k} 件)】
{doc_texts}

上記の回答が参照ドキュメントと比較して、どの程度正確・妥当かを1〜5点で評価してください。
1：ほぼ全てが不正確
2：不正確な点が多い
3：おおよそ正しいがいくつか誤り/不足がある
4：ほとんど正確
5：非常に正確・参照ドキュメントと整合

理由の説明は不要なので、数値のみを出力してください。
"""
        eval_response = evaluator_llm.call_as_llm(eval_prompt)

        # スコアの抽出 (雑に数字を拾う)
        match = re.search(r"(\d+)", eval_response)
        if match:
            score = int(match.group(1))
        else:
            score = None

        results.append(
            {"question": q, "answer": ans, "score": score, "eval_raw": eval_response}
        )

    # スコアの平均など
    valid_scores = [r["score"] for r in results if r["score"] is not None]
    avg_score = sum(valid_scores) / len(valid_scores) if valid_scores else 0
    return results, avg_score


In [22]:
def get_answers(non_rag_llm, rag_chain, questions: List[str]):
    rag_answers = []
    non_rag_answers = []

    for q in questions:
        # RAGなし回答
        non_rag_res = non_rag_llm.call_as_llm(q)
        non_rag_answers.append(non_rag_res)

        # RAGあり回答
        rag_res = rag_chain.run(q)
        rag_answers.append(rag_res)

    return non_rag_answers, rag_answers

In [25]:
def evaluate_rag_vs_nonrag(
    questions: List[str],
    non_rag_llm: ChatOpenAI,
    rag_chain: RetrievalQA,
    evaluation_retriever,
    evaluator_llm: ChatOpenAI,
    top_k=3,
):
    """
    RAGなし vs RAGあり の回答を取得し、それぞれを評価する。
    """
    # 1. 回答を取得
    non_rag_answers, rag_answers = get_answers(non_rag_llm, rag_chain, questions)

    # 2. RAGなし回答を評価
    print("\n=== Evaluating Non-RAG answers ===")
    non_rag_eval, non_rag_avg = score_answers_by_docs(
        evaluator_llm=evaluator_llm,
        questions=questions,
        answers=non_rag_answers,
        retriever=evaluation_retriever,
        top_k=top_k,
    )
    print(f"Non-RAG Average Score: {non_rag_avg:.2f}")

    # 3. RAGあり回答を評価
    print("\n=== Evaluating RAG answers ===")
    rag_eval, rag_avg = score_answers_by_docs(
        evaluator_llm=evaluator_llm,
        questions=questions,
        answers=rag_answers,
        retriever=evaluation_retriever,
        top_k=top_k,
    )
    print(f"RAG Average Score: {rag_avg:.2f}")

    # 4. 結果比較
    print("\n=== Comparison ===")
    print(f"- Non-RAG Score: {non_rag_avg:.2f}")
    print(f"- RAG Score    : {rag_avg:.2f}")
    improvement = rag_avg - non_rag_avg
    print(f"-> Improvement : {improvement:.2f}")

    # 5. 必要なら JSON などに書き出し
    results_all = {
        "non_rag_eval": non_rag_eval,
        "rag_eval": rag_eval,
        "non_rag_avg": non_rag_avg,
        "rag_avg": rag_avg,
        "improvement": improvement,
    }
    with open("evaluation_rag_vs_nonrag.json", "w", encoding="utf-8") as f:
        json.dump(results_all, f, ensure_ascii=False, indent=2)

    return results_all


In [29]:
rag_llm = ChatOpenAI(
    model_name="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0
)

non_rag_llm = ChatOpenAI(
    model_name="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0
)

rag_chain = RetrievalQA.from_chain_type(
    llm=rag_llm,
    chain_type="stuff",
    retriever=retriever,
)

evaluation_retriever = retriever

evaluator_llm = ChatOpenAI(
    model_name="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0
)

questions = [
    "LAMMPSはどのような分子動力学シミュレーションが可能ですか？",
    "LAMMPSのインストール方法を教えてください。",
    "LAMMPSのGPUアクセラレーションはどうやって有効にするの？",
]

results = evaluate_rag_vs_nonrag(
    questions=questions,
    non_rag_llm=non_rag_llm,
    rag_chain=rag_chain,
    evaluation_retriever=evaluation_retriever,
    evaluator_llm=evaluator_llm,
    top_k=3,
)



=== Evaluating Non-RAG answers ===
Non-RAG Average Score: 3.33

=== Evaluating RAG answers ===
RAG Average Score: 4.33

=== Comparison ===
- Non-RAG Score: 3.33
- RAG Score    : 4.33
-> Improvement : 1.00
